In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from threading import Thread

In [16]:
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36' }

response = requests.get(f'https://ana-slo.com/2024-09-08-ロイヤル網走店-data/' , headers=headers)
print(response.status_code)
# soup = BeautifulSoup(response.text, 'html.parser')
# div = soup.find_all('div', id='all_data_block')
# tr = div[0].find_all('tr')

403


In [12]:
response

<Response [403]>

In [9]:
response.text

'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}body.no-js .loading-spinner{visibility:hidden}body.theme-dark{background-color:#222;color:#d9d9d9}body.theme-dark a{color:#fff}body.theme-dark a:hover{color:#ee730a;text-decoration:underline}body.theme-dark .lds-ring div{border-color:#999 transparent transparent}body.theme-dark .font-red{color:#b20f03}body.theme-dar

In [4]:
import ana_slo 
a = ana_slo.ana_slo('北海道', 'ロイヤル網走店')
a

IndexError: list index out of range

In [1]:
import re
pattern = r'^[^,]+,[^,]+$'
user_message = '東京都,アミューズ浅草店'
if  re.match(pattern, user_message):
    country,storename = user_message.split(',')
    print(country,storename)

東京都 アミューズ浅草店


In [39]:
#the main function

class ana_slo_crawler:

    def __init__(self, country, store):
        response = requests.get(f'https://ana-slo.com/ホールデータ/{country}/{store}-データ一覧/')
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('div' , id= 'table')

        #找到所有有 a 的日期
        a = table.find_all('a')
        a_list = []
        for date in a:
            a_list.append(date.text.strip()[:10].replace('/', '-'))
        self.a_list = a_list

        self.store = store 

        response = requests.get(f'https://ana-slo.com/{self.a_list[0]}-{store}-data/')
        soup = BeautifulSoup(response.text, 'html.parser')
        div = soup.find_all('div', id='all_data_block')
        tr = div[0].find_all('tr')
        columns_slot = tr[0].text.strip().split('\n')
        columns_slot.append('DATE')
        self.columns_slot = columns_slot
        #['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', '合成確率', 'BB確率', 'RB確率','DATE']

    def get_slot_data_14(self):  

        def _get_slot_data(date , store):
            nonlocal data_list

            response = requests.get(f'https://ana-slo.com/{date}-{store}-data/')
            soup = BeautifulSoup(response.text, 'html.parser')

            div = soup.find_all('div', id='all_data_block')
            tr = div[0].find_all('tr')
            columns_slot = tr[0].text.strip().split('\n')

            for td in tr[1:len(tr)]:
                data = td.text.strip().split('\n')
                if len(data)==len(columns_slot):
                    data.append(date)
                    data_list.append(data)

        data_list = []
        ths = [None] * len(self.a_list[:14])
        for i in range(len(self.a_list[:14])):
            ths[i] = Thread(target=_get_slot_data, args=(self.a_list[i],self.store ), daemon=True)
            ths[i].start()

        for i in range(len(self.a_list[:14])):
            ths[i].join()   
            
        return data_list

In [40]:
data = ana_slo_crawler('北海道','ロイヤル網走店')
print(data.a_list )
print(data.store )
print(data.columns_slot)


['2024-09-06', '2024-08-28', '2024-08-26', '2024-08-18', '2024-08-16', '2024-08-08', '2024-08-06', '2024-07-28', '2024-07-26', '2024-07-18', '2024-07-16', '2024-07-08', '2024-07-06', '2024-06-28', '2024-06-26', '2024-06-18', '2024-06-16', '2024-06-08', '2024-06-06', '2024-05-28', '2024-05-26', '2024-05-18', '2024-05-16', '2024-05-08', '2024-05-06', '2024-04-28', '2024-04-26', '2024-04-18', '2024-04-16', '2024-04-08', '2024-04-06', '2024-03-28', '2024-03-26', '2024-03-24', '2024-03-21', '2024-03-18', '2024-03-16', '2024-03-08', '2024-03-06', '2024-02-28', '2024-02-26', '2024-02-18', '2024-02-16', '2024-02-08', '2024-02-06', '2024-01-28', '2024-01-26', '2024-01-18', '2024-01-16', '2024-01-08', '2024-01-06', '2023-12-28', '2023-12-26', '2023-12-18', '2023-12-16', '2023-12-08', '2023-12-06', '2023-11-28', '2023-11-26', '2023-11-18', '2023-11-16', '2023-11-08', '2023-11-06', '2023-10-28', '2023-10-26', '2023-10-18', '2023-10-16', '2023-10-08', '2023-10-06', '2023-09-28', '2023-09-18', '2023

In [41]:
df_all = pd.DataFrame(data.get_slot_data_14(), columns=data.columns_slot)
df_all.head()

,機種名,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,DATE
0,アイムジャグラーEX-TP,482,"1,590",-236,5,5,1/159.0,1/318.0,1/318.0,2024-07-26
1,アイムジャグラーEX-TP,484,"2,066",-836,5,4,1/229.6,1/413.2,1/516.5,2024-07-26
2,アイムジャグラーEX-TP,486,"1,812","-1,165",2,6,1/226.5,1/906.0,1/302.0,2024-07-26
3,アイムジャグラーEX-TP,488,"3,068",-593,10,3,1/236.0,1/306.8,1/1022.7,2024-07-26
4,アイムジャグラーEX-TP,490,"1,452",+178,6,2,1/181.5,1/242.0,1/726.0,2024-07-26


In [42]:

#------------------------------------------------
#用差枚總數排行
df_all['差枚'] = df_all['差枚'].fillna(0)
df_all['差枚'] = df_all['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
top10_coins = df_all.groupby('台番号')['差枚'].sum().sort_values(ascending=False).head(10)
df_all['win'] = df_all['差枚'].apply(lambda x: 1 if x > 0 else 0)
#top10.index 篩選df_all['台號'] == top10.index
df_top10_coins = df_all[df_all['台番号'].isin(top10_coins.index)]
#計算top10的WIN RATE
top10_coins_win_rate = df_top10_coins.groupby('台番号')['win'].mean().sort_values(ascending=False).head(10)
df_final_coins = pd.merge(top10_coins, top10_coins_win_rate, on='台番号', how='left')
print(df_final_coins)

#------------------------------------------------
#計算['win']裡的1的比例 groupby 台號
win_rate_top10  = df_all.groupby('台番号')['win'].mean().sort_values(ascending=False).head(10)
#merge對應的差枚
top10_win_rate_coins = df_all[df_all['台番号'].isin(win_rate_top10.index)]
top10_win_rate_coins = top10_win_rate_coins.groupby('台番号')['差枚'].sum()
df_final_win_rate = pd.merge( win_rate_top10,top10_win_rate_coins , on='台番号', how='left')
print(df_final_win_rate)

        差枚       win
台番号                 
520  32534  0.500000
412  29940  0.357143
457  26748  0.785714
442  21070  0.642857
427  19125  0.571429
426  18217  0.642857
501  16008  0.428571
459  15621  0.500000
429  15605  0.428571
445  13926  0.428571
          win     差枚
台番号                 
457  0.785714  26748
426  0.642857  18217
474  0.642857   5904
461  0.642857   5902
442  0.642857  21070
535  0.571429   4755
492  0.571429   1435
542  0.571429   2809
544  0.571429   1056
434  0.571429  13309


C:\Users\文傑\AppData\Local\Temp\ipykernel_364600\507179861.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_all['差枚'] = df_all['差枚'].str.replace(',', '').str.replace('+', '').astype(int)


以下分析

In [ ]:
df_all['差枚'] = df_all['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
#差枚欄位改為int
df_all['差枚'].astype(int)

In [ ]:
#小丑
dic = {1 : 1/163.8 ,
2 : 1/159.1
,3 : 1/148.6
,4 : 1/135.2
,5 : 1/126.8
,6 : 1/114.6}

In [ ]:
dic.values()

In [ ]:
def safe_division(x):
    try:
        return eval(x) if '/' in x else float(x)
    except ZeroDivisionError:
        return 0  # 或者其他你希望在分母為零時返回的值

df_all['合成確率'] = df_all['合成確率'].apply(safe_division)
df_all['合成確率'].astype(float)

In [ ]:
min(dic.values(), key=lambda x:abs(x-0.006))

In [ ]:
#最接近dic 的value給予其key 例如最接近 1/163.8 的值為1 , 1/159.1 的值為2
def find_nearest(dic , value):
    nearest = min(dic.values(), key=lambda x:abs(x-value))
    for k, v in dic.items():
        if v == nearest:
            return k


In [ ]:
#新增一個欄位並套用find_nearest function
df_all['合成確率_rank'] = df_all['合成確率'].apply(lambda x : find_nearest(dic , x))

In [ ]:
マイジャグラーV = df_all[df_all['機種名']=='マイジャグラーV'].sort_index(ascending=True)

In [ ]:
マイジャグラーV.to_csv('マイジャグラーV.csv', index=False)

In [ ]:
df_all.to_csv('slot_data_マルハン石和店_20240514.csv', index=False)

In [ ]:
#test for the data image

respinse = requests.get('https://ana-slo.com/2024-05-07-%e3%83%9e%e3%83%ab%e3%83%8f%e3%83%b3%e7%9f%b3%e5%92%8c%e5%ba%97-data/')
soup = BeautifulSoup(respinse.text, 'html.parser')

div = soup.find_all('div', id='tab02_0')
#找到div裡的script  
script = div[0].find_all('script')
len(script)
#提取script裡的 x_value , y_value
x_value = script[0].text.split('x_value = ')[1].split(';')[0]
y_value = script[0].text.split('y_value = ')[1].split(';')[0]
#用x_value , y_value 劃出折線圖
import matplotlib.pyplot as plt
x = json.loads(x_value)
y = json.loads(y_value)
plt.plot(x, y)
plt.show()

